In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, f1_score, accuracy_score, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import warnings


In [2]:
student_df = pd.read_csv('data/stud.csv')
student_df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [3]:
X = student_df.drop(['math_score'], axis = 1)
y = student_df['math_score']
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [4]:
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [8]:
numerical_features = X.select_dtypes(exclude = 'object').columns
cat_features = X.select_dtypes(include = 'object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
scaler = StandardScaler()
encoder = OneHotEncoder()

preprocessor = ColumnTransformer([
    ("OHE", encoder, cat_features),
    ("SS", scaler, numerical_features)
])

In [9]:
X = preprocessor.fit_transform(X)

In [12]:
X.shape

(1000, 19)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

((800, 19), (200, 19), (800,), (200,))

In [40]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(true, predicted)
    r2 = r2_score(true, predicted)
    return mae, rmse, r2

In [41]:
models = {
    'linear regression' : LinearRegression(),
    'ridge' : Ridge(),
    'lasso' : Lasso(),
    'Random Forest' : RandomForestRegressor(),
    'xgb' : XGBRegressor()
}

model_list = []
r2 = []

for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    X_train_mae, X_train_rmse, X_train_r2 = evaluate_model(y_train, y_train_pred)
    X_test_mae, X_test_rmse, X_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('model perfomance for training: ')
    print("mean abs error : {:.4f}".format(X_train_mae))
    print("rmse : {:.2f}".format(X_train_rmse.iloc[0]))
    print("r2 : {:.4f}".format(X_train_r2))


    print("------------------------")

    print('model perfomance for testing: ')
    print("mean abs error : {:.4f}".format(X_test_mae))
    print("rmse : {:.4f}".format(X_test_rmse.iloc[0]))
    print("r2 : {:.4f}".format(X_test_r2))
    r2.append(X_test_r2)

    print("===============")
    print('\n')

linear regression
model perfomance for training: 
mean abs error : 4.2685
rmse : 7.87
r2 : -15.0041
------------------------
model perfomance for testing: 
mean abs error : 4.2184
rmse : 9.5394
r2 : -12.9885


ridge
model perfomance for training: 
mean abs error : 4.2650
rmse : 7.87
r2 : -15.0041
------------------------
model perfomance for testing: 
mean abs error : 4.2111
rmse : 9.5394
r2 : -12.9885


lasso
model perfomance for training: 
mean abs error : 5.2063
rmse : 7.87
r2 : -15.0041
------------------------
model perfomance for testing: 
mean abs error : 5.1579
rmse : 9.5394
r2 : -12.9885


Random Forest
model perfomance for training: 
mean abs error : 1.8346
rmse : 7.87
r2 : -15.0041
------------------------
model perfomance for testing: 
mean abs error : 4.6093
rmse : 9.5394
r2 : -12.9885


xgb
model perfomance for training: 
mean abs error : 0.6875
rmse : 7.87
r2 : -15.0041
------------------------
model perfomance for testing: 
mean abs error : 5.0577
rmse : 9.5394
r2 : -12

In [43]:
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 88.01
